# Starter


#### Hello readers,

#### In this notebook, we will try to explore different ways of developing the machine learning model for this competition.

#### Few points to consider before we begin :

* This notebook basically focuses on the CNN-based approaches.
* We cover the topics like Transfer learning and Fine-tuning the pre-trained CNN models.
* *THIS IS A DEMONSTRATION NOTEBOOK THEREFORE WE ONLY USE THE 100 IMAGES OTU OF ~9900 ( FOR FAST PROCESSING )*.
* Note that because of the above reason the performance of the model is not the true indicator of its abilities.


#### A brief about what we are going to cover in this notebook 

* Data analysis and preprocessing. 
* Transfer learning ( pre-trained model selection ).
* Fine-tuning the pre-trained model.


#### A brief about the approach we are taking :

* As the score ranges from 1 to 100, i.e every image has a score between 1 to 100. We just need to find the score of an image. One possible approach is to treat the problem as a multiclass classification problem, where we have an image and we want to classify it as one of the 100 categories. 

#### Let’s begin, Hope you enjoy this notebook! 


<center style="font-family:verdana;"><h1 style="background: #f4c2c2 ;">_</h1></center>

<center><h1>Data analysis and preprocessing  </h1></center>

### Let's have a look at train.csv

In [ ]:
import pandas as pd

train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv',usecols = ['Id','Pawpularity'])
train_df.head()

#### we have image id and its respective score in the train.csv

### Let's have a look at some images from the train folder 

In [ ]:
import cv2
import random

image_1 = cv2.imread('../input/petfinder-pawpularity-score/train/' +random.choice(train_df['Id']) + '.jpg' )
image_2 = cv2.imread('../input/petfinder-pawpularity-score/train/' +random.choice(train_df['Id']) + '.jpg' )

print(image_1.shape)
print(image_2.shape)

#### We can see the problem here, The input to the model is an image and the size of the input has to be constant, but in the dataset, we don't have all the images with the fixed dimensions, Well we can easily crop the image to the desired size.

#### which brings us to our question what should be the size of the input image ?, It’s very important that we find the correct size. If the image size is very small we are simply losing a lot of information and if the size is very high, that adds extra non-sence information (padding ) as well as a lot of processing overhead ( high ram consumption ). 

#### Here we will iterate through all the image’s shapes and take the average of all and use that as the final image size. 


In [ ]:
import numpy as np
import cv2

from PIL import Image


path = '../input/petfinder-pawpularity-score/train/'

images = []
labels = []

image_heights = []
image_widths = []

count = 1
Max_examples = 100


for id_,y in zip(train_df['Id'],train_df['Pawpularity']):
    
    img = cv2.imread(path + id_ + '.jpg', cv2.COLOR_BGR2RGB)
    
    images.append(img)
    
    image_heights.append(img.shape[0])
    image_widths.append(img.shape[1])
    
    labels.append(y-1)
    
    if count == Max_examples:
        break
    
    count+=1

avg_h = sum(image_heights)//len(image_heights)
avg_w = sum(image_widths)//len(image_widths)

    
print(f'average image hieght is {avg_h}')

print(f'average image width is {avg_w}')


### Now let's convert the images in the desired format 

In [ ]:
X_train = []

for img in images:
    
    img = cv2.resize(img,(avg_h,avg_w),interpolation = cv2.INTER_AREA)
    img = np.array(img)
    img = img.astype('float32')
    img /= 255 
    
    X_train.append(img)
    
X_train = np.array(X_train)
print(X_train.shape)

In [ ]:
IMG_SHAPE = (avg_w,avg_h,3)

### Let's prepare the y_train in the correct format 

In [ ]:
Y_train = np.array(labels)

Y_train = Y_train.reshape(100,1)
print(Y_train.shape)

### DONE

In [ ]:
#end

## Important NOTE   

#### The above method for preparing the data is very basic and high ram consuming, If we do not have access to high ram machines we should use the Image data generator, which loads the batch of images instead of loading all the images in the ram.

#### Image data generator is an efficient approach but relatively complicated, here we are trying to explore and thus I have added this simple approach instead. 


<center style="font-family:verdana;"><h1 style="background: #f4c2c2 ;">_</h1></center>

<center><h1>Transfer learning ( pre-trained model selection ) </h1></center>

#### Transfer learning is the process of using the models that have been trained on the very large image datasets, And utilize the learned information (weights ) and model architecture for the give use cases. 

#### You can learn more about transfer learning here: https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/

#### let's start 


In [ ]:
import tensorflow as tf

#### There are a lot of pre-trained models that we can use, different models have different properties and we can select the model based on the use case. But ultimately it’s a matter of trying, i.e we have to try and find the best model for our use-cases out of all the pre-trained models. 

#### you can find the pre-trained models here: https://keras.io/api/applications/

#### here we will be using resnet50 as the pre-trained model, you can learn more about resnet50 here: https://keras.io/api/applications/resnet/#resnet50-function


In [ ]:
Pretrained_model = tf.keras.applications.resnet50.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

#### Now that we have loaded the pre-trained model, We are just gonna use the learned information from this model by integrating the pre-trained model with our model.


## Feature extraction : Freeze the convolutional base

#### freezing the layers prevents the layer’s weight from being updated during the training. Because we want to use that information as it is.

In [ ]:
Pretrained_model.trainable = False

In [ ]:
Pretrained_model.summary()

#### Now that the pre-trained model is ready we can start developing the main model here

In [ ]:
# model 
Inputs = tf.keras.Input(IMG_SHAPE)

x = Pretrained_model(Inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dropout(0.2)(x)
x =  tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dropout(0.1)(x)
Outputs = tf.keras.layers.Dense(100)(x)

Model = tf.keras.Model(Inputs, Outputs)

### simple explanation of the above model 

* Image enters into the model as a multidimensional array
* We pass the image to pretrained mode 
* Pre-trained model returns the vectorized representation of the image 
* these vector passes through the Feedforward network 
* final output is generated 


#### Now We are going to compile the model and set the hyperparameters for our model, here we will be using the adam optimizer and SparseCategoricalCrossentropy as a loss function.

In [ ]:
Optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
Loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
Metrics=['accuracy']

Model.compile(optimizer=Optimizer,
              loss = Loss,
              metrics = Metrics)

In [ ]:
Model.summary()

#### defining the callbacks is very important, we can decide the behavior of the training.
#### Tune the bellow parameters as needed.

In [ ]:
callbacks =[tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
),
tf.keras.callbacks.ModelCheckpoint(
    filepath = './model', monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)]

#### As this is the demonstration we will only train this model for 10 epochs  

## Let's start the training ;)


In [ ]:
history = Model.fit(X_train, Y_train, epochs=10, 
                    validation_split = 0.1, verbose = 1,callbacks = callbacks)

## Let's see the performance  

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')


### As we can see the performance is not so good but the model's performance can be increased significantly if

* we use 100% of the data ( in the above example we used ~ 10% of the data )
* we add more layers
* We try different pre-trained model
* We do long training of the model

#### In the next section, we will try to improve the performance 


In [ ]:
#end

<center style="font-family:verdana;"><h1 style="background: #f4c2c2 ;">_</h1></center>

<center><h1>Fine-tuning the pre-trained mode</h1></center>

#### Fine Tunning is the process of updating the weights of the pre-trained model in order to increase the accuracy, you can learn more about fine-tuning the pre-trained model here: https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/ 

#### Similar to transfer learning in fine-tuning the pre-trained model is used, but here we allow the pre-trained model’s layer's weights to get updated during the training.

#### We can eighter train all the layers or we can train only a few layers.


### Un-freeze some layers of the model

In [ ]:
Pretrained_model.trainable = True

In [ ]:
len( Pretrained_model.layers)

#### we can see that there are 175 layers in the pre-trained model, here we will start fine-tuning the model from the 100th layer.


In [ ]:

start_ = 100

for layer in Pretrained_model.layers[:start_]:
    layer.trainable =  False

### importent note: 
    
#### now we are training a much larger model than before, we need to be careful while setting the learning rate of the model, the learning rate should be lower otherwise model could overfit very quickly.


In [ ]:

Optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
Loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
Metrics=['accuracy']

Model.compile(optimizer=Optimizer,
              loss = Loss,
              metrics = Metrics)

In [ ]:
Model.summary()

## Let's start the training.

In [ ]:
history = Model.fit(X_train, Y_train, epochs=10, 
                    validation_split = 0.1, verbose = 1,callbacks = callbacks)

## Let's have look at performance 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')


In [ ]:
#end

### Note :

* We have seen the different approaches that we can use here in the context of CNN only.
* One other approach that we can use is transformers, have a look here: https://keras.io/examples/vision/image_classification_with_vision_transformer/

* The metadata can also be used for the training of the model.
* Simple techniques like hyperparameter tunning, long training, image data extraction ... can be used to improve the model performance.




## Thank you for reading, please share your thoughts/suggestions  